In [ ]:
#!g1.1
!git clone https://github.com/nerel-ds/NEREL

In [ ]:
!unzip NEREL-v1.0.zip -d NEREL

In [342]:
#!g1.1
## Read the data# Reading files
from collections import namedtuple
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import nltk

Ann = namedtuple('annotation', ['tag', 'start1', 'end1', 'start2', 'end2', 'text'])
Rel = namedtuple('relationship', ['tag', 'arg1', 'arg2'])

def read_files(folder):
    filenames = sorted(set(e[:e.rfind('.')] for e in os.listdir(folder)))[1:]
    
    texts, entities, relationships  = [], [], []
    for file in tqdm(filenames):
       # if (not file): continue
        path1 = os.path.join(folder, file+'.txt')
        path2 = os.path.join(folder, file+'.ann')
        if (not os.path.exists(path2)):
            print(f'{path2} not found')
            continue
            #with open(path2, 'w'):
            #    pass
            
        with open(path1, 'r', encoding="utf8") as text, open(path2, 'r', encoding="utf8") as ann:
            texts.append(text.read())

            file_entities = {}
            file_relationship = []
            regex_ent = r'T(?P<id>\d+)\s(?P<tag>\w+)\s(?P<start1>\d+) (?P<end1>\d+)(;(?P<start2>\d+) (?P<end2>\d+))?\s(?P<text>.*)'
            regex_rel = r'R(?P<id>\d+)\s(?P<tag>\w+)\sArg1:T(?P<arg1>\d+) Arg2:T(?P<arg2>\d+)'
            
            
            for row in sorted(ann.readlines(), reverse=True):
                #print(row)
                match_ent = re.match(regex_ent, row)
                match_rel = re.match(regex_rel, row)
                if (match_ent):
                    res = match_ent.groupdict()
                    res['start1'] = int(res['start1'])
                    res['end1'] = int(res['end1'])
                    if (res['start2'] is not None):
                        res['start2'] = int(res['start2'])
                        res['end2'] = int(res['end2'])
                    id = res.pop('id')
                    file_entities[id] = Ann(**res)
                elif (match_rel):
                    try:
                        res = match_rel.groupdict()
                        res['arg1'] = file_entities[res['arg1']]
                        res['arg2'] = file_entities[res['arg2']]
                        id = res.pop('id')
                        file_relationship.append(Rel(**res))
                    except KeyError as e:
                        print(f'not found T{e} row={row}')
                else:
                    print(f'incorrect format in: row={row} file={file}')
            entities.append(file_entities)
            relationships.append(file_relationship)
    entities = [sorted(e.values(), key = lambda x: (x.start1, x.end1)) for e in entities]
    return texts, entities, relationships, filenames

In [343]:
#!g1.1
folder = 'NEREL/NEREL-v1.0/train'
texts, entities, relationships, filenames = read_files(folder)

incorrect format in: row=По словам очевидцев пешехо
 file=21013_text


100%|██████████| 745/745 [00:01<00:00, 384.82it/s]


In [344]:
#!g1.1
def in_range(range1, range2):
    # range1 in range2
    if range1[0] is None: return True
    if range2[0] is None: return False
    return int(range2[0]) <= int(range1[0]) and int(range2[1]) >= int(range1[1])

def is_nested_anns(ent1: Ann, ent2: Ann):
    # ent 1 in ent 2
    res = True
    ent1_range1 = (ent1.start1, ent1.end1)
    ent1_range2 = (ent1.start2, ent1.end2)
    ent2_range1 = (ent2.start1, ent2.end1)
    ent2_range2 = (ent2.start2, ent2.end2)
    #print(f'{ent1_range1=} {ent1_range2=} {ent1_range2=} {ent2_range2=}')
    res = res and (in_range(ent1_range1, ent2_range1) or in_range(ent1_range1, ent2_range2))
    res = res and (in_range(ent1_range2, ent2_range1) or in_range(ent1_range2, ent2_range2))
    return res


def is_nested_anns2(ent1: Ann, ent2: Ann):
    return is_nested_anns(ent1,ent2) or is_nested_anns(ent2, ent1)

def is_nested(rel: Rel):
    return is_nested_anns(rel.arg1, rel.arg2) or is_nested_anns(rel.arg2, rel.arg1)

![](https://i.imgur.com/tgDfc8i.png)             | ![](https://i.imgur.com/oWa5vWo.png)
:-------------------------:|:-------------------------:


In [345]:
#!g1.1
%pip install transformers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


# Dataset

In [346]:
#!g1.1
import os
import torch
import time
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from tqdm.notebook import tqdm
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from matplotlib import pyplot as plt
import random
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [347]:
#!g1.1
from transformers import BertForSequenceClassification, BertTokenizer

In [348]:
#!g1.1
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [349]:
#!g1.1
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

In [350]:
#!g1.1
MAX_LENGTH = 100

In [351]:
#!g1.1
# import nltk
# nltk.download('punkt')
# #!g1.1
# sents = nltk.sent_tokenize(texts[0])
# len(' '.join(sents))

In [360]:
#!g1.1
class Binary_Tag:
    def __getitem__(self, x):
        if (x == 'None'): return 0
        else: return 1

# tags = list(set(e.tag for e in sum(relationships,[]))) + ['None'] # ids --> string tag
reversed_tags = Binary_Tag() # string tag --> ids
entity_tags = list(set(e.tag for e in sum(entities,[]))) 

def tok(text1, text2):
    # longest entity is 36 tokens long
    res = tokenizer.encode_plus(text1, text2,
                      max_length = MAX_LENGTH,
                      pad_to_max_length = True,
                      return_tensors = 'pt',
                      )
    return res['input_ids'][0], res['attention_mask'][0], res['token_type_ids'][0]

def order(e1, e2):
    if (len(e1.text) > len(e2.text)):
        return e1, e2
    else:
        return e2, e1


class MyDataset(torch.utils.data.Dataset):
    def __init__(self, folder):
        texts, entities, relationships, filenames = read_files(folder)
        
        pre_dataset = []

        for text_id in range(len(texts)):
            relationships_nested = {(e.arg1, e.arg2):e for e in relationships[text_id] if is_nested(e)}
            nes = []
            for i in range(len(entities[text_id])):
                # O(n^2) eeeeeeeeee
                for j in range(i+1, len(entities[text_id])):
                    if (is_nested_anns2(entities[text_id][i], entities[text_id][j])):
                        nes.append((entities[text_id][i], entities[text_id][j]))
            for e in nes:
                if (e in relationships_nested):
                    pre_dataset.append((*order(*e), relationships_nested[e].tag, text_id))
                elif ((e[1], e[0]) in relationships_nested):
                    pre_dataset.append((*order(*e), relationships_nested[(e[1], e[0])].tag, text_id))
                else:
                    pre_dataset.append((*order(*e), 'None', text_id))

        data = []
        for e1, e2, tag, text_id in tqdm(pre_dataset):
            
            data.append((reversed_tags[tag], entity_tags.index(e1.tag), entity_tags.index(e2.tag), tag, *tok(e1.text, e2.text)))
        
        self.data = data
    
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return len(self.data)
    

In [361]:
#!g1.1
train_data = MyDataset('NEREL/NEREL-v1.0/train')
train_dl = DataLoader(train_data, shuffle=True, batch_size=32)

dev_data = MyDataset('NEREL/NEREL-v1.0/dev')
dev_dl = DataLoader(dev_data, batch_size=32)

incorrect format in: row=По словам очевидцев пешехо
 file=21013_text






In [ ]:
#!g1.1
next(iter(dev_dl))[1]

In [ ]:
#!g1.1
next(iter(dev_dl))[1].size()

# Training

In [362]:
#!g1.1
from transformers import BertTokenizer, BertModel
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained(
            "DeepPavlov/rubert-base-cased-sentence", # Use the 12-layer BERT model, with an uncased vocab.
            output_attentions = False, # Whether the model returns attentions weights.
            output_hidden_states = False, # Whether the model returns all hidden-states.
            max_length = MAX_LENGTH
            )
        self.dropout = nn.Dropout(0.1)
        self.emb = nn.Embedding(29, 10)
        self.fc = nn.Sequential(
            nn.Linear(768 + 10*2, 200),
            nn.LeakyReLU(True),
            nn.Linear(200, 200),
            nn.LeakyReLU(True),
            nn.Linear(200, 2)
            )
        
    def forward(self, ent1, ent2, input_ids, attention_mask, token_type_ids, labels):
        ent1 = self.emb(ent1)
        ent2 = self.emb(ent2)
        out = self.bert(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask)[1]
        out = self.dropout(out)
        out = torch.cat((out, ent1, ent2), dim=1)
        out = self.fc(out)
        criterion = nn.CrossEntropyLoss()
        loss = criterion(out.view(-1, 2), labels.view(-1))
        return loss, out
    
        # requires reconfiguration of hyper parameters
#         criterion = nn.BCEWithLogitsLoss()
#         out = self.fc(out).view(-1)
#         loss = criterion(out, labels)
#         return loss, torch.stack((-torch.sigmoid(out), torch.sigmoid(out)), dim = 1)

In [363]:
#!g1.1
model = MyModel()
model.to(device)

MyModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
   

In [364]:
#!g1.1
def calculate_accuracy(y_pred, y):
    return sum(torch.argmax(y_pred, axis = 1) == y)*1.0/len(y)

def train():
    model.train()
    running_loss = 0
    epoch_accuracy = 0

    pbar = tqdm(enumerate(train_dl), total = len(train_dl))
    for i, (labels, tag1, tag2, true_tag, input_ids, attention_mask, token_type_ids) in pbar:
        
        labels = labels.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        tag1 = tag1.to(device)
        tag2 = tag2.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(tag1, tag2,
                    input_ids, 
                    token_type_ids=token_type_ids,
                    attention_mask=attention_mask, 
                    labels=labels)
        
        loss = outputs[0]
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        epoch_accuracy += calculate_accuracy(outputs[1], labels).item()
        running_loss += loss.item()
        pbar.set_description(f'training: running_loss = {running_loss/(i+1.0):.4f} accuracy = {epoch_accuracy/(i+1.0):.4f},')
    
    scheduler.step(running_loss/(i+1))
    print(f'train loss= {running_loss/(i+1):.4f} \n train accuracy = {epoch_accuracy/(i+1):.4f},')

    
from sklearn.metrics import f1_score
def test():
    with torch.no_grad():
        model.eval()
        running_loss = 0
        epoch_accuracy = 0

#         dev_dl = DataLoader( MyDataset('NEREL/NEREL-v1.0/train'), batch_size = 32)

        pbar = tqdm(enumerate(dev_dl), total = len(dev_dl))

        y_true = []
        y_pred = []

        for i, (labels, tag1, tag2, true_tag, input_ids, attention_mask, token_type_ids) in pbar:
            labels = labels.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            tag1 = tag1.to(device)
            tag2 = tag2.to(device)

            outputs = model(tag1, tag2,
                        input_ids, 
                        token_type_ids=token_type_ids,
                        attention_mask=attention_mask, 
                        labels=labels)

            loss = outputs[0]
#             calculate_accuracy(outputs[1], labels)
            y_true += [e.item() for e in labels]
            y_pred += [e.item() for e in torch.argmax(outputs[1], axis = 1)]
            
            running_loss += loss.item()
            epoch_accuracy += calculate_accuracy(outputs[1], labels).item()
            # pbar.set_description(f'testing: running_loss = {running_loss/(i+1):.4f} accuracy = {epoch_accuracy/(i+1):.4f},')
#     print(y_true, y_pred)
    print(f'test loss= {running_loss/(i+1):.4f} \n test accuracy = {epoch_accuracy/(i+1):.4f}, F1 = {f1_score(y_true, y_pred)}')

In [365]:
#!g1.1
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate 
                  eps = 5e-8#1e-8 # args.adam_epsilon 
                )
scheduler = ReduceLROnPlateau(optimizer, patience = 3, cooldown = 1, factor = 0.5)

In [366]:
#!g1.1
seed_val = 2021

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
for epoch in range(1, 21):
    print(f'epoch = {epoch}')
    print('lr=', optimizer.param_groups[0]['lr'])
    train()
    test()

epoch = 1
lr= 5e-05

train loss= 0.3627 
 train accuracy = 0.8396,

test loss= 0.3311 
 test accuracy = 0.8715, F1 = 0.8370731707317074
epoch = 2
lr= 5e-05

train loss= 0.2420 
 train accuracy = 0.9090,

test loss= 0.3460 
 test accuracy = 0.8533, F1 = 0.7957446808510638
epoch = 3
lr= 5e-05

train loss= 0.1886 
 train accuracy = 0.9358,

test loss= 0.3626 
 test accuracy = 0.8692, F1 = 0.8232848232848233
epoch = 4
lr= 5e-05

train loss= 0.1644 
 train accuracy = 0.9444,

test loss= 0.4458 
 test accuracy = 0.8540, F1 = 0.8049281314168377
epoch = 5
lr= 5e-05

train loss= 0.1435 
 train accuracy = 0.9521,

test loss= 0.4441 
 test accuracy = 0.8544, F1 = 0.8073022312373226
epoch = 6
lr= 5e-05

train loss= 0.1261 
 train accuracy = 0.9597,

test loss= 0.5093 
 test accuracy = 0.8601, F1 = 0.8229571984435797
epoch = 7
lr= 5e-05

train loss= 0.1049 
 train accuracy = 0.9662,

test loss= 0.5901 
 test accuracy = 0.8582, F1 = 0.8185404339250493
epoch = 8
lr= 5e-05

train loss= 0.0952 
 train 

In [367]:
#!g1.1
torch.save(model.state_dict(), 'models/Set2_BERT_Binary_tags_model2_dataset2')

In [373]:
#!g1.1
with torch.no_grad():
    model.eval()
    running_loss = 0
    epoch_accuracy = 0

#     dev_dl = DataLoader( MyDataset('NEREL/NEREL-v1.0/dev'), batch_size = 1)

    pbar = tqdm(enumerate(dev_dl), total = len(dev_dl))

    y_true = []
    y_pred = []
    tags = []

    for i, (labels, tag1, tag2, true_tag, input_ids, attention_mask, token_type_ids) in pbar:
        labels = labels.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        tag1 = tag1.to(device)
        tag2 = tag2.to(device)

        outputs = model(tag1, tag2,
                        input_ids, 
                        token_type_ids=token_type_ids,
                        attention_mask=attention_mask, 
                        labels=labels)

        loss = outputs[0]
#             calculate_accuracy(outputs[1], labels)
        y_true += [e.item() for e in labels]
        y_pred += [e.item() for e in torch.argmax(outputs[1], axis = 1)]
        tags += [(entity_tags[tag1], entity_tags[tag2])]
        
        running_loss += loss.item()
        epoch_accuracy += calculate_accuracy(outputs[1], labels).item()
        if (torch.argmax(outputs[1], axis = 1) != labels):
            print('expected:', labels.detach().cpu().item())
            print('found:', torch.argmax(outputs[1], axis = 1).detach().cpu().item(), '(', outputs[1].detach().cpu().numpy() ,')')
            print(f'{tags[-1][0]} ---> {tags[-1][0]} : {true_tag[0]}')
            print(tokenizer.decode(input_ids[0].detach().cpu().numpy()).replace(r'[PAD]', ''))
            print()
            
print(f'test loss= {running_loss/(i+1):.4f} \n test accuracy = {epoch_accuracy/(i+1):.4f}, F1 = {f1_score(y_true, y_pred)}')

expected: 0
found: 1 ( [[-2.8255627  2.780704 ]] )
ORGANIZATION ---> ORGANIZATION : None
[CLS] Строгановское высшее художественное училище [SEP] Строгановское [SEP]                                                                                          

expected: 0
found: 1 ( [[-2.2936409  2.3161287]] )
PROFESSION ---> PROFESSION : None
[CLS] главным менеджером Международного музыкального фестиваля « Ереванские перспективы » [SEP] Ереванские перспективы [SEP]                                                                                    

expected: 0
found: 1 ( [[-2.854251  2.921277]] )
EVENT ---> EVENT : None
[CLS] Международного музыкального фестиваля « Ереванские перспективы [SEP] Ереванские [SEP]                                                                                        

expected: 1
found: 0 ( [[ 2.3023918 -2.165439 ]] )
EVENT ---> EVENT : ALTERNATIVE_NAME
[CLS] Международного музыкального фестиваля « Ереванские перспективы [SEP] Ереванские перспективы [SEP]     

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:859: UserWarning: The following variables cannot be serialized: pbar
  warnings.warn(message)


In [375]:
#!g1.1
from collections import Counter
l = []
for true, pred, t in zip(y_true, y_pred, tags):
    if (true != pred):
        l.append((t[0], t[1]))
Counter(l).most_common()

[(('PROFESSION', 'ORGANIZATION'), 23),
 (('ORGANIZATION', 'COUNTRY'), 13),
 (('PROFESSION', 'COUNTRY'), 12),
 (('ORGANIZATION', 'ORGANIZATION'), 12),
 (('EVENT', 'EVENT'), 9),
 (('EVENT', 'LOCATION'), 9),
 (('LAW', 'LAW'), 7),
 (('PROFESSION', 'PROFESSION'), 6),
 (('WORK_OF_ART', 'PERSON'), 6),
 (('EVENT', 'DISEASE'), 5),
 (('DISEASE', 'DISEASE'), 5),
 (('ORGANIZATION', 'STATE_OR_PROVINCE'), 5),
 (('FACILITY', 'PERSON'), 5),
 (('EVENT', 'PERSON'), 5),
 (('ORGANIZATION', 'PERSON'), 4),
 (('LAW', 'COUNTRY'), 4),
 (('PROFESSION', 'STATE_OR_PROVINCE'), 4),
 (('ORGANIZATION', 'CITY'), 4),
 (('ORGANIZATION', 'LOCATION'), 3),
 (('WORK_OF_ART', 'FACILITY'), 3),
 (('PROFESSION', 'PERSON'), 3),
 (('ORGANIZATION', 'PROFESSION'), 3),
 (('EVENT', 'COUNTRY'), 3),
 (('PROFESSION', 'EVENT'), 2),
 (('LAW', 'STATE_OR_PROVINCE'), 2),
 (('PENALTY', 'MONEY'), 2),
 (('WORK_OF_ART', 'CITY'), 2),
 (('PROFESSION', 'CITY'), 2),
 (('PERSON', 'PERSON'), 2),
 (('EVENT', 'PROFESSION'), 2),
 (('EVENT', 'WORK_OF_ART'